<a href="https://colab.research.google.com/github/lancedsouza/Webscraping_projects/blob/main/FinalCode_for_extractingGemdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code for downloading pdf

In [ ]:
# Final Code for downloading pdfs
import os
import time
import io
import base64
from PIL import Image
import pytesseract

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

# =============================
# CONFIGURATION
# =============================
DOWNLOAD_DIR = os.path.join(os.getcwd(), "downloaded_pdfs")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

VIEW_URL = "https://gem.gov.in/view_contracts"
SEARCH_TERM = "Hot air gun"
FROM_DATE = "01-Jan-2024"  # format: dd-MMM-yyyy

# =============================
# SELENIUM SETUP
# =============================
def setup_driver():
    chrome_options = Options()
    prefs = {
        "download.default_directory": DOWNLOAD_DIR,
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_ext": True
    }
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--start-maximized")
    return webdriver.Chrome(options=chrome_options)


def remove_popups(driver):
    for pid in ("gemmy-text-item", "flox-text-item", "gemmyIcon"):
        driver.execute_script(f"document.getElementById('{pid}')?.remove();")


def solve_captcha(el):
    """Capture the captcha image and extract text using OCR."""
    png = el.screenshot_as_png
    img = Image.open(io.BytesIO(png)).convert("L")
    return pytesseract.image_to_string(img).strip()


def scroll_all_contracts(driver, pause_time=1, stable_limit=7):
    """Scroll page until all contracts are loaded."""
    last_count = 0
    stable_runs = 0
    step_px = 400

    while stable_runs < stable_limit:
        while True:
            scroll_top = driver.execute_script("return window.scrollY + window.innerHeight")
            scroll_height = driver.execute_script("return document.body.scrollHeight")
            if scroll_top + 10 >= scroll_height:
                break
            driver.execute_script("window.scrollBy(0, arguments[0]);", step_px)
            time.sleep(pause_time)

        elems = driver.find_elements(By.XPATH, "//a[contains(@onclick,'openCap')]")
        count = len(elems)
        print(f"🔍 {count} contracts loaded so far…")

        if count > last_count:
            last_count = count
            stable_runs = 0
        else:
            stable_runs += 1
            print(f"⚠️ No new contracts (stable {stable_runs}/{stable_limit})")

        time.sleep(pause_time)

    print(f"✅ Finished scrolling; total contracts found: {last_count}")
    return elems

# =============================
# MAIN LOGIC
# =============================
def main():
    driver = setup_driver()
    wait = WebDriverWait(driver, 20)
    processed = set()

    try:
        driver.get(VIEW_URL)
        time.sleep(2)
        remove_popups(driver)

        # Select category
        wait.until(EC.element_to_be_clickable((By.ID, 'select2-buyer_category-container'))).click()
        fld = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'select2-search__field')))
        fld.send_keys(SEARCH_TERM + Keys.ENTER)
        time.sleep(1)
        remove_popups(driver)

        # Set from date
        d, m, y = FROM_DATE.split('-')
        wait.until(EC.element_to_be_clickable((By.ID, 'from_date_contract_search1'))).click()
        Select(driver.find_element(By.CLASS_NAME, "ui-datepicker-month")).select_by_visible_text(m)
        Select(driver.find_element(By.CLASS_NAME, "ui-datepicker-year")).select_by_visible_text(y)
        wait.until(EC.element_to_be_clickable((By.XPATH, f"//a[text()='{int(d)}']"))).click()
        time.sleep(1)

        # First CAPTCHA
        cap1_el = wait.until(EC.presence_of_element_located((By.ID, 'captchaimg1')))
        cap1_text = solve_captcha(cap1_el)
        driver.find_element(By.ID, 'captcha_code1').send_keys(cap1_text)
        wait.until(EC.element_to_be_clickable((By.ID, 'searchlocation1'))).click()
        time.sleep(2)

        # Load contracts
        links = scroll_all_contracts(driver)
        onclicks = [el.get_attribute('onclick') for el in links]
        print(f"➡️ Total contracts to process: {len(onclicks)}")

        for idx, js in enumerate(onclicks, 1):
            print(f"➡️ Contract {idx}/{len(onclicks)}")
            if js in processed:
                print("   ⏭ Already processed, skipping.")
                continue

            driver.execute_script(js)
            time.sleep(1)

            try:
                cap2_el = wait.until(EC.presence_of_element_located((By.ID, 'captchaimg')))
                cap2_text = solve_captcha(cap2_el)
                cap_input = wait.until(EC.element_to_be_clickable((By.ID, 'captcha_code')))
                cap_input.clear()
                cap_input.send_keys(cap2_text)
            except Exception as e:
                print("⚠️ Could not solve CAPTCHA automatically. You can solve it manually now.")
                input("🔓 Press ENTER after you’ve manually entered CAPTCHA and clicked Submit.")

            # Submit
            try:
                wait.until(EC.element_to_be_clickable((By.ID, 'modelsbt'))).click()
            except:
                print("⚠️ Submit button not found or clickable.")

            # Pause for manual download
            print("🧍 Waiting for manual CAPTCHA input and download...")
            input("🔓 After clicking 'Download', press ENTER to continue.")
            print("✅ Continuing…")

            processed.add(js)
            time.sleep(1)

        print(f"\n🚀 Completed. Total contracts processed: {len(processed)}")

    finally:
        driver.quit()

if __name__ == '__main__':
    main()


Code for extracting pdf data

In [ ]:
#Final Code for extracting data from PDF
import os
import re
import fitz  # PyMuPDF
import pandas as pd

def normalize_hindi_first(text: str) -> str:
    """
    Swap occurrences where Hindi (Devanagari) appears before English separated by '|',
    e.g., "अनुबंध|Contract" → "Contract|अनुबंध", so English label is first.
    """
    pattern = re.compile(r'([\u0900-\u097F\s]+)\|([\w\s]+)')
    return pattern.sub(r'\2|\1', text)

def clean_text(text: str) -> str:
    """
    Clean raw text extracted from PDF:
    1. Normalize Hindi-first labels
    2. Rejoin split email tokens (remove spaces/newlines after '@')
    3. Remove Devanagari characters
    4. Replace '|' separators with ' : '
    5. Replace control/newline/tab chars with space
    6. Remove unwanted special chars except those useful for IDs, emails, numbers
    7. Normalize multiple spaces to single space
    """
    # 1. Normalize Hindi-first labels
    text = normalize_hindi_first(text)

    # 2. Rejoin split email tokens: remove spaces/newlines after '@'
    text = re.sub(r'@\s+', '@', text)

    # 3. Remove Devanagari entirely
    text = re.sub(r'[\u0900-\u097F]', '', text)

    # 4. Replace vertical bars '|' with ' : '
    text = re.sub(r'\s*\|\s*', ' : ', text)

    # 5. Replace control characters/newlines/tabs with space
    text = re.sub(r'[\n\r\t\x0b\x0c]', ' ', text)

    # 6. Remove unwanted special characters but keep those useful for IDs, emails, numbers
    #    Allows: word chars (letters/digits/underscore), @ . , : / & ( ) - + % # and whitespace
    text = re.sub(r'[^\w@.,:/&()\-+%#\s]', '', text)

    # 7. Normalize multiple spaces to single space
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extract all text from the PDF at pdf_path using PyMuPDF, then clean it.
    """
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
    except Exception as e:
        print(f"Error opening {pdf_path}: {e}")
        return ""
    cleaned = clean_text(text)
    # Debug: inspect a snippet of cleaned text
    # print(f"=== Cleaned text for {os.path.basename(pdf_path)} ===\n{cleaned[:500]}\n")
    return cleaned

def extract_section(text: str, start_label: str, end_label: str) -> str:
    """
    Extract substring between start_label and next occurrence of end_label.
    If end_label is empty or not found, returns everything after start_label.
    Matches labels case-insensitively, allows optional colon, strips leading colons/spaces.
    """
    if not start_label or start_label not in text:
        return ""
    if end_label:
        pattern = re.compile(
            rf"{re.escape(start_label)}\s*(?:\:)?(.*?)(?={re.escape(end_label)}\s*(?:\:)?|$)",
            re.IGNORECASE | re.DOTALL
        )
    else:
        pattern = re.compile(
            rf"{re.escape(start_label)}\s*(?:\:)?(.*)$",
            re.IGNORECASE | re.DOTALL
        )
    m = pattern.search(text)
    if not m:
        return ""
    sec = m.group(1).strip()
    sec = sec.lstrip(": ").strip()
    # Debug: inspect the isolated section
    # print(f"=== Section '{start_label}' content ===\n{sec[:300]}\n")
    return sec

def extract_buyer(section: str) -> dict:
    """
    Extract fields from Buyer Details section, using flexible patterns:
    - buyer_designation
    - buyer_contact_no
    - buyer_email
    - buyer_gstin
    - buyer_address
    """
    fields = {
        "buyer_designation": None,
        "buyer_contact_no": None,
        "buyer_email": None,
        "buyer_gstin": None,
        "buyer_address": None
    }
    if not section:
        return fields
    section = section.lstrip(": ").strip()

    # DESIGNATION: up to next Contact No or Email ID
    m = re.search(r"Designation\b.*?:\s*(.+?)(?=(Contact\s*No\b|Email\s*ID\b|$))",
                  section, re.IGNORECASE)
    if m:
        val = m.group(1).strip().rstrip(":").strip()
        fields["buyer_designation"] = val or None

    # CONTACT NO: look for first sequence of digits/hyphens after "Contact No"
    m = re.search(r"Contact\s*No\b.*?([0-9][0-9\-\s]+)", section, re.IGNORECASE)
    if m:
        val = m.group(1).strip().rstrip("- ").strip()
        fields["buyer_contact_no"] = val or None

    # EMAIL ID: look for email pattern after "Email ID"
    m = re.search(
        r"Email\s*ID\b.*?([A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,})",
        section, re.IGNORECASE
    )
    if m:
        email = m.group(1).strip().rstrip("_ ").strip()
        fields["buyer_email"] = email or None

    # GSTIN: strict 15-character GSTIN pattern: 2 digits then 13 alphanumeric
    # Example pattern: r"\b[0-9]{2}[A-Z0-9]{13}\b"
    m = re.search(r"GSTIN\b.*?\b([0-9]{2}[A-Za-z0-9]{13})\b", section, re.IGNORECASE)
    if m:
        val = m.group(1).strip().upper()
        fields["buyer_gstin"] = val or None

    # ADDRESS: until next known label or end
    m = re.search(
        r"Address\b.*?:(.*?)(?=(Financial Approval Detail\b|Paying Authority Details\b|$))",
        section, re.IGNORECASE | re.DOTALL
    )
    if m:
        addr = re.sub(r'\s+', ' ', m.group(1)).strip()
        fields["buyer_address"] = addr or None

    return fields

def extract_seller(section: str) -> dict:
    """
    Extract fields from Seller Details section, using flexible patterns:
    - seller_id
    - seller_name
    - seller_contact
    - seller_email
    - seller_gstin
    - seller_address
    """
    fields = {
        "seller_id": None,
        "seller_name": None,
        "seller_contact": None,
        "seller_email": None,
        "seller_gstin": None,
        "seller_address": None
    }
    if not section:
        return fields
    section = section.lstrip(": ").strip()

    # GeM Seller ID
    m = re.search(r"GeM Seller ID\b.*?(\S+)", section, re.IGNORECASE)
    if m:
        fields["seller_id"] = m.group(1).strip()

    # Company Name
    m = re.search(r"Company Name\b.*?:\s*(.+?)(?=(Contact\s*No\b|Email\s*ID\b|$))",
                  section, re.IGNORECASE)
    if m:
        val = m.group(1).strip().rstrip(":").strip()
        fields["seller_name"] = val or None

    # Contact No
    m = re.search(r"Contact\s*No\b.*?([0-9][0-9\-\s]+)", section, re.IGNORECASE)
    if m:
        val = m.group(1).strip().rstrip("- ").strip()
        fields["seller_contact"] = val or None

    # Email ID
    m = re.search(
        r"Email\s*ID\b.*?([A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,})",
        section, re.IGNORECASE
    )
    if m:
        email = m.group(1).strip().rstrip("_ ").strip()
        fields["seller_email"] = email or None

    # GSTIN: strict 15-character pattern
    m = re.search(r"GSTIN\b.*?\b([0-9]{2}[A-Za-z0-9]{13})\b", section, re.IGNORECASE)
    if m:
        val = m.group(1).strip().upper()
        fields["seller_gstin"] = val or None

    # Address
    m = re.search(r"Address\b.*?:(.*?)(?=(MSME Registration\b|GST/TAX\b|$))",
                  section, re.IGNORECASE | re.DOTALL)
    if m:
        addr = re.sub(r'\s+', ' ', m.group(1)).strip()
        fields["seller_address"] = addr or None

    return fields

def extract_product(section: str) -> dict:
    """
    Extract product-related fields; unchanged from previous patterns.
    """
    fields = {
        "product_name": None,
        "product_brand": None,
        "product_category": None,
        "product_model": None,
        "ordered_quantity": None,
        "unit": None,
        "unit_price": None,
        "tax_bifurcation": None,
        "price_inclusive": None,
        "total_order_value": None
    }
    if not section:
        return fields

    # Product Name
    m = re.search(r"Product Name\b.*?:\s*(.+?)(?=\s*Brand\b)", section, re.IGNORECASE)
    if m:
        fields["product_name"] = m.group(1).strip()

    # Brand
    m = re.search(r"Brand\b.*?:\s*(.+?)(?=\s*Brand Type\b)", section, re.IGNORECASE)
    if m:
        fields["product_brand"] = m.group(1).strip()

    # Category
    m = re.search(r"Category Name & Quadrant\b.*?:\s*(.+?)(?=\s*Model\b)", section, re.IGNORECASE)
    if m:
        fields["product_category"] = m.group(1).strip()

    # Model
    m = re.search(r"Model\b.*?:\s*(.+?)(?=\s*HSN Code\b)", section, re.IGNORECASE)
    if m:
        fields["product_model"] = m.group(1).strip()

    # After HSN Code: ordered_quantity, unit, unit_price, tax_bifurcation, price_inclusive
    m = re.search(
        r"HSN Code\b.*?([\d,]+)\s+(\w+)\s+([\d,]+)\s+([A-Za-z0-9]+)\s+([\d,]+)",
        section, re.IGNORECASE
    )
    if m:
        fields["ordered_quantity"] = m.group(1).strip()
        fields["unit"] = m.group(2).strip()
        fields["unit_price"] = m.group(3).strip()
        fields["tax_bifurcation"] = m.group(4).strip()
        fields["price_inclusive"] = m.group(5).strip()

    # Total Order Value
    m = re.search(r"Total Order Value\b.*?\(?in INR\)?\s*:\s*([\d,]+)", section, re.IGNORECASE)
    if m:
        fields["total_order_value"] = m.group(1).strip()

    return fields

def process_folder(pdf_dir: str, output_excel: str):
    """
    Iterate over PDFs in pdf_dir, extract fields, and save to Excel.
    """
    all_rows = []
    for fname in os.listdir(pdf_dir):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(pdf_dir, fname)
        text = extract_text_from_pdf(path)
        if not text:
            continue

        buyer_sec = extract_section(text, "Buyer Details", "Seller Details")
        seller_sec = extract_section(text, "Seller Details", "Product Details")
        product_sec = extract_section(text, "Product Details", "")

        # Debug: inspect sections if needed
        # print(f"--- {fname} Buyer Section ---\n{buyer_sec}\n")
        # print(f"--- {fname} Seller Section ---\n{seller_sec}\n")

        row = {"filename": fname}
        row.update(extract_buyer(buyer_sec))
        row.update(extract_seller(seller_sec))
        row.update(extract_product(product_sec))
        all_rows.append(row)

    if not all_rows:
        print("No PDFs found or no data extracted in folder:", pdf_dir)
        return
    df = pd.DataFrame(all_rows)
    try:
        df.to_excel(output_excel, index=False)
        print(f"Saved extracted data to {output_excel}")
    except Exception as e:
        print(f"Error saving to Excel {output_excel}: {e}")

if __name__ == "__main__":
    # Adjust pdf_folder path as needed
    pdf_folder = r"C:\Users\Wishes Lawrence\downloaded_pdfs"
    output_file = os.path.join(pdf_folder, "extracted_contracts1_hotairgun.xlsx")
    process_folder(pdf_folder, output_file)
